# Algorithm prototyping

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
import pyproj
from pyproj import CRS, Transformer

In [2]:
%matplotlib inline

In [10]:
df = pd.read_csv("data\extracted\dom1l-fp_32349_5660_1_nw.csv")

In [11]:
df.head()

,x,y,z,lat,lon
0,349000.0,5660017.01,43.84,51.071779,6.844637
1,349000.0,5660019.82,50.58,51.071805,6.844636
2,349000.0,5660034.90,60.41,51.071940,6.844630
3,349000.0,5660057.71,43.85,51.072145,6.844620
4,349000.0,5660058.17,43.83,51.072149,6.844620


In [12]:
df = df.set_index(["lat", "lon"])

In [13]:
df = df.sort_index()

In [14]:
df.head()

x           y      z
lat       lon                                   
51.071627 6.844651  349000.44  5660000.07  43.98
51.071628 6.844686  349002.94  5660000.03  43.99
          6.844684  349002.76  5660000.08  51.07
          6.844686  349002.90  5660000.12  47.75
          6.844707  349004.37  5660000.08  47.99

In [15]:
lat_q, lon_q = 51.078534, 6.844748

In [ ]:
df.index.serachsorted([lat_q, lon_q])